In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# File to Load 
customer_data_to_load = Path("Resources/customer_churn.csv")

# Read Data File and store into Pandas DataFrames
churn_df = pd.read_csv(customer_data_to_load)

# View the dataset
churn_df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42,11066.80,0,7.22,8,8/30/2013 7:00,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41,11916.22,0,6.50,11,8/13/2013 0:38,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38,12884.75,0,6.67,12,6/29/2016 6:20,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42,8010.76,0,6.71,10,4/22/2014 12:43,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37,9191.58,0,5.56,9,1/19/2016 15:31,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [2]:
# Changing column names to make data cleaner
churn_df = churn_df.rename(columns = {'Total_Purchase': 'Total Purchase', 'Account_Manager' : 'Account Manager', 'Num_Sites': 'Number of Websites Used', 'Onboard_date' : 'Onboard Date'})

# View updated column names
churn_df.head()

,Names,Age,Total Purchase,Account Manager,Years,Number of Websites Used,Onboard Date,Location,Company,Churn
0,Cameron Williams,42,11066.80,0,7.22,8,8/30/2013 7:00,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41,11916.22,0,6.50,11,8/13/2013 0:38,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38,12884.75,0,6.67,12,6/29/2016 6:20,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42,8010.76,0,6.71,10,4/22/2014 12:43,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37,9191.58,0,5.56,9,1/19/2016 15:31,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [3]:
# Summarize the data
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Names                    900 non-null    object 
 1   Age                      900 non-null    int64  
 2   Total Purchase           900 non-null    float64
 3   Account Manager          900 non-null    int64  
 4   Years                    900 non-null    float64
 5   Number of Websites Used  900 non-null    int64  
 6   Onboard Date             900 non-null    object 
 7   Location                 900 non-null    object 
 8   Company                  900 non-null    object 
 9   Churn                    900 non-null    int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 70.4+ KB


In [4]:
# Drop the non-beneficial ID columns.
churn_df = churn_df.drop(["Names","Onboard Date","Location","Company"],1)
churn_df.head()

,Age,Total Purchase,Account Manager,Years,Number of Websites Used,Churn
0,42,11066.80,0,7.22,8,1
1,41,11916.22,0,6.50,11,1
2,38,12884.75,0,6.67,12,1
3,42,8010.76,0,6.71,10,1
4,37,9191.58,0,5.56,9,1


In [5]:
# Determine the number of unique values in each column.
churn_df.nunique()

Age                         36
Total Purchase             900
Account Manager              2
Years                      418
Number of Websites Used     12
Churn                        2
dtype: int64

In [6]:
# Split our preprocessed data into our features and target arrays
y = churn_df['Churn'].values

# Separate the X variable, the features
x = churn_df.drop(columns='Churn').values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(x_train[0])
hidden_nodes_layer1 =  500
hidden_nodes_layer2 = 300

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               3000      
                                                                 
 dense_1 (Dense)             (None, 300)               150300    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 153601 (600.00 KB)
Trainable params: 153601 (600.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(x_train_scaled,y_train,epochs=100)

Epoch 1/100
22/22 [==============================] - 1s 7ms/step - loss: 0.4767 - accuracy: 0.8341
Epoch 2/100
22/22 [==============================] - 0s 6ms/step - loss: 0.4332 - accuracy: 0.8341
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 0.4078 - accuracy: 0.8341
Epoch 4/100
22/22 [==============================] - 0s 8ms/step - loss: 0.3625 - accuracy: 0.8385
Epoch 5/100
22/22 [==============================] - 0s 7ms/step - loss: 0.3554 - accuracy: 0.8622
Epoch 6/100
22/22 [==============================] - 0s 10ms/step - loss: 0.3120 - accuracy: 0.8770
Epoch 7/100
22/22 [==============================] - 0s 7ms/step - loss: 0.2810 - accuracy: 0.8844
Epoch 8/100
22/22 [==============================] - 0s 14ms/step - loss: 0.2628 - accuracy: 0.8948
Epoch 9/100
22/22 [==============================] - 0s 8ms/step - loss: 0.2744 - accuracy: 0.8874
Epoch 10/100
22/22 [==============================] - 0s 7ms/step - loss: 0.2746 - accuracy: 0.8919
Epoch 1

22/22 [==============================] - 0s 8ms/step - loss: 0.2471 - accuracy: 0.9022
Epoch 83/100
22/22 [==============================] - 0s 9ms/step - loss: 0.2493 - accuracy: 0.8978
Epoch 84/100
22/22 [==============================] - 0s 6ms/step - loss: 0.2476 - accuracy: 0.9007
Epoch 85/100
22/22 [==============================] - 0s 12ms/step - loss: 0.2576 - accuracy: 0.8948
Epoch 86/100
22/22 [==============================] - 0s 8ms/step - loss: 0.2504 - accuracy: 0.9037
Epoch 87/100
22/22 [==============================] - 0s 14ms/step - loss: 0.2510 - accuracy: 0.8963
Epoch 88/100
22/22 [==============================] - 0s 11ms/step - loss: 0.2479 - accuracy: 0.9022
Epoch 89/100
22/22 [==============================] - 0s 12ms/step - loss: 0.2465 - accuracy: 0.9081
Epoch 90/100
22/22 [==============================] - 0s 8ms/step - loss: 0.2506 - accuracy: 0.9007
Epoch 91/100
22/22 [==============================] - 0s 8ms/step - loss: 0.2471 - accuracy: 0.9037
Epoch 92/

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.3094 - accuracy: 0.8711 - 212ms/epoch - 26ms/step
Loss: 0.3094363212585449, Accuracy: 0.8711110949516296


In [12]:
# Export our model to HDF5 file
nn.save('Churn_Optimization_3.h3')

INFO:tensorflow:Assets written to: Churn_Optimization_3.h3\assets


INFO:tensorflow:Assets written to: Churn_Optimization_3.h3\assets
